# **Code Review_Version1** 

## **Backbone : VGG16** 

## **Import Library**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

import time
import random
import copy

## **Define Model**

In [ ]:
"""# **1) Model define**
### trans_VGG에서 사용할 함수인 conv_2 define
"""

def conv_2(in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),# Model define
        nn.Conv2d(out_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

def conv_3(in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),# Model define
        nn.Conv2d(out_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

In [ ]:
# conv_2 함수 : 2개의 합성곱 레이어와 Relu 활성화 함수로 구성된 block 정의 후 max pooling을 통해 크기 감소
# conv_3 함수 : 4개의 합성곱 레이어와 Relu 활성화 함수로 구성된 block 정의 후 max pooling을 통해 크기 감소
# Conv2d(): 3 x 3 필터(커널), padding=1 인 2차원 합성곱 레이어
# Relu(): 활성화 함수 Relu를 통해 비선형성 추가
# MaxPool2d(): max pooling을 통해 입력값의 크기를 절반으로 줄임

# 과적합 방지를 위한 Dropout 제안
# 이미지의 크기가 감소하는 특징 존재

## **Define trans_VGG class**

In [ ]:
class trans_VGG(nn.Module):
    def __init__(self, base_dim):
        super(trans_VGG, self).__init__()
        self.feature = nn.Sequential(
            conv_2(3, base_dim),
            conv_2(base_dim, base_dim*2),
            conv_2(base_dim*2, base_dim*4),
            conv_3(base_dim*4, base_dim*8),
            conv_3(base_dim*8, base_dim*8)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(base_dim*8*7*7, base_dim*4*7*7),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(base_dim*4*7*7, base_dim*2*7*7),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(base_dim*2*7*7, base_dim*7*7)
        )
        for param in self.parameters():
            param.requires_grad = True

    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

In [ ]:
# self.feature -> 특성 추출
# 차원 수가 2배씩 점차 증가하는 특징 존재

# self.fc_layer -> 완전 연결층과 같이 1차원으로 펼치는 작업
# layer의 크기를 점차 줄여가며 출력값을 생성한다.
 
# foward 함수
# 입력 이미지를 특성 추출 레이어에 통과시켜 특성 맵을 추출 후, 1차원 벡터로 변환
# 변환된 벡터를 완전 연결층에 통과시켜 최종 예측을 수행하는 함수

- Hyper_paremeter : Learning rate, momentum, weight decay 등은 논문의 Hyper peremeter value로 초기화


In [ ]:
import torch.nn.init as init

seed = time.time()

def custom_init_weights(m):
  if seed is not None:
    torch.manual_seed(seed)
  if isinstance(m, torch.nn.Linear) and m.weight is not None:
    init.normal_(m.weight, mean=1, std=0.01)
    if m.bias is not None:
      init.constant_(m.bias, 0)

model = trans_VGG(base_dim=64)

loss = nn.BCELoss()
optimizer =torch.optim.SGD(model.parameters(), lr = 0.01,momentum = 0.9, weight_decay = 0.0005)
scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=10, factor=0.1, verbose=True)

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.RandomCrop(224)])

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# custom_init_weights 함수
# 모델의 각 레이어가 torch.nn.Linear 타입일 때, 
# 가중치(w)를 평균 1 표준편차 0.01인 정규분포로 초기화, 편향(b)은 0으로 초기화

# trans_VGG class 객체 생성 -> 생성자에 의해 모델의 layer 설정

# loss function 정의 BCELoss(): 이진 교차 엔트로피 소실 함수
# optimizer: 확률적 경사하강법 사용, 학습률 0.01, momentum=0.9(빠른 수렴), weight_decay=0.0005: 가중치 감쇠(정규화)
# ReduceLROnPlateau: 모델의 성능 향상이 10번의 epoch 동안 없을 때 학습률 10배 감소 + 성능 최대화 시 학습률 조정

# transform: 이미지의 픽셀 값(0~255)을 0과 1 사이의 값으로 정규화 및 224 x 224 크기의 Random Crop 적용

## **Import Dataset**

In [ ]:
import os
from PIL import Image
import numpy as np
from torch.utils.data import Dataset

# Project 3 폴더 경로
project_folder = '/content/drive/MyDrive/Project3'

image = []
label = []

# Project 3 폴더 내부의 세부 폴더를 확인하고 이미지와 라벨 데이터 생성
for subdir, _, files in os.walk(project_folder):
    for file in files:
        # 이미지 파일인지 확인
        if file.endswith(('png', 'jpg', 'jpeg')):
            image_path = os.path.join(subdir, file)
            image.append(image_path)

            # 이미지가 속한 세부 폴더의 이름을 라벨로 사용
            label_name = os.path.basename(subdir)
            label.append(label_name)

indices = np.random.permutation(len(image))
IMAGE = [image[i] for i in indices]
LABEL = [label[i] for i in indices]

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert('RGB')
        image = transforms.RandomCrop(224)(image)
        image = transforms.ToTensor()(image)

        return image, label

BATCH_SIZE = 1

TRAINING_image = []
TRAINING_label = []
TEST_image = []
TEST_label = []

for i in range(0,80):
  for j in range(0,20):
    for k in range(0,2):
      TRAINING_image.append(image[200*j+i+k])
      TRAINING_label.append(label[200*j+i+k])

for i in range(80,100):
  for j in range(0,20):
    for k in range(0,2):
      TEST_image.append(image[200*j+i+k])
      TEST_label.append(label[200*j+i+k])

train_dataset = CustomDataset(TRAINING_image, TRAINING_label, transform = transform)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE,num_workers=2)
test_dataset = CustomDataset(TEST_image, TEST_label, transform = transform)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE,num_workers=2)

## **Training**

In [ ]:
"""# **3) TRAINING**"""

EPOCH = 80 # 반복횟수

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(DEVICE)

# 시간 측정
start_time = time.time()
train_acc_lst, test_acc_lst = [],[]

for epoch in range(EPOCH):
  model.train() # 학습모드
  correct_pred, num_examples = 0, 3200 # 정확하게 예측한 샘플 수 (초기값), 학습 데이터셋의 크기
  for i, (_image1, _label1) in enumerate(train_loader): # mini-batch 반복을 통해 이미지에 대한 tensor 입력
    image1 = _image1.to(DEVICE)
    label1 = _label1[0]
    vector1_tensor = model(image1)

    if (i == 0): #Exception Case
      image2 = image1
      label2 = label1
      vector2_tensor = vector1_tensor

    # 두 이미지의 코사인 유사도 측정
    similarity =  F.cosine_similarity(vector1_tensor, vector2_tensor, dim= -1)
    scaled_similarity = torch.sigmoid(similarity)

    # 유사도 기준 = 0.5 
    if label1 == label2 and scaled_similarity.item() > 0.5: # 두 이미지의 라벨이 동일한 경우
        correct_pred += 1
    elif label1 != label2 and scaled_similarity.item() < 0.5: # 두 이미지의 라벨이 다른 경우
        correct_pred += 1

    if label1 == label2:
      target_vector = [1]
    else :
      target_vector = [0]

    # 역전파 및 가중치 업데이트
    target_tensor = torch.tensor(target_vector).float()
    target_tensor = target_tensor.to(DEVICE)
    optimizer.zero_grad()
    cost = loss(scaled_similarity, target_tensor)
    cost.backward()
    optimizer.step()

    # 40개의 batch마다 학습의 진행 상태 및 cost 출력
    if not i % 40:
      print (f'Epoch: {epoch:03d}/{EPOCH:03d} | '
            f'Batch {i:03d}/{len(train_loader):03d} |'
             f' Cost: {cost:.4f}')

    #연산량 감소를 위한 텐서 재활용
    image2 = image1.clone()
    label2 = label1
    vector2_tensor = vector1_tensor.detach().clone()

elapsed = (time.time() - start_time)/60
print(f'Total Training Time: {elapsed:.2f} min')

In [ ]:
# model: VGG-model
# 가중치 초기화 및 손실 함수, 옵티마이저 정의
# 이미지 크기 변환 및 random crop 활용

# 배치 크기 조정 및 배치 정규화 추가를 통한 성능 개선 고려
# 옵티마이저 함수 변경을 통한 성능 개선 고려
# 추가적인 이미지 증강 고려